In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [4]:
# Dados Entrada

ger = ['Produtor A', 'Produtor B']
Ng = len(ger)
pmax_ter = [100, 80] # MW
c_ener_ger = [12, 20]

loads = ['Consumidor 1', 'Consumidor 2']
Nc = len(loads)
demanda_energia = [100, 50]
c_ener_load = [40, 35]

c = [0, 0.5, 2, 5]

In [5]:
# Criar Modelo Concreto
model = ConcreteModel()

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener_ger = Param(model.ger, initialize={ger[i]: c_ener_ger[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: pmax_ter[i] for i in range(Ng)})

#Cargas:
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: demanda_energia[i] for i in range(Nc)})
model.c_ener_load = Param(model.loads, initialize={loads[i]: c_ener_load[i] for i in range(Nc)})

In [6]:
# Declarar Variáveis
model.P_ger = Var(model.ger, model.loads, domain=NonNegativeReals)
model.P_load = Var(model.loads, model.ger, domain=NonNegativeReals)

In [7]:
#Restrições:
def balanco(model, ger, load):
    return model.P_load[load, ger] == model.P_ger[ger, load]
model.balanco = Constraint(model.ger, model.loads, rule=balanco)

def limite_ger(model, ger):
    return sum(model.P_ger[ger, i] for i in model.loads) <= model.pmax_ter[ger]
model.limite_ger = Constraint(model.ger, rule=limite_ger)

def limite_load(model, load):
    return sum(model.P_load[load, i] for i in model.ger) <= model.dem[load]
model.limite_load = Constraint(model.loads, rule=limite_load)

In [8]:
#Função Objetivo:
def objective_function(model):
    energy_sell = sum(model.c_ener_ger[i] * model.P_ger[i, j] for i in model.ger for j in model.loads)
    energy_buy = sum(model.c_ener_load[j] * model.P_load[j, i] for j in model.loads for i in model.ger)

    return energy_buy - energy_sell

model.objective = Objective(rule=objective_function, sense=maximize)

In [9]:
solver = SolverFactory('glpk')
# solver = SolverFactory('ipopt',executable='/content/ipopt')
model.dual = Suffix(direction=Suffix.IMPORT)
resultados = solver.solve(model, tee=False)

In [10]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.objective), '\n')

#Energias Vendidas

for i in model.ger:
    for j in model.loads:
        print(f'Energia Vendida do {i} para o {j}:', value(model.P_ger[i,j]))

for j in model.loads:
    for i in model.ger:
        print(f'Energia Comprada do {i} para o {j}:', value(model.P_load[j,i]))

#Venda total dos geradores em $:
for i in model.ger:
    total = sum(value(model.P_ger[i,j]) * model.c_ener_ger[i] for j in model.loads)
    print(f'Venda total do {i}:', total)

#Compra total das cargas em $:
for j in model.loads:
    total = sum(value(model.P_load[j,i]) * model.c_ener_load[j] for i in model.ger)
    print(f'Compra total do {j}:', total)

# Impressão dos preços sombra
for (ger, load) in model.balanco:
    print(f"Dual para restrição de balanço {ger}-{load}: {model.dual[model.balanco[ger, load]]}")

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 3550.0 

Energia Vendida do Produtor A para o Consumidor 1: 100.0
Energia Vendida do Produtor A para o Consumidor 2: 0.0
Energia Vendida do Produtor B para o Consumidor 1: 0.0
Energia Vendida do Produtor B para o Consumidor 2: 50.0
Energia Comprada do Produtor A para o Consumidor 1: 100.0
Energia Comprada do Produtor B para o Consumidor 1: 0.0
Energia Comprada do Produtor A para o Consumidor 2: 0.0
Energia Comprada do Produtor B para o Consumidor 2: 50.0
Venda total do Produtor A: 1200.0
Venda total do Produtor B: 1000.0
Compra total do Consumidor 1: 4000.0
Compra total do Consumidor 2: 1750.0
Dual para restrição de balanço Produtor A-Consumidor 1: 20.0
Dual para restrição de balanço Produtor A-Consumidor 2: 20.0
Dual para restrição de balanço Produtor B-Consumidor 1: 20.0
Dual para restrição de balanço Produtor B-Consumidor 2: 20.0
